In [1]:
import numpy as np
from scipy.stats import norm

# Parâmetros do problema
S = 50               # Preço da ação
K1 = 50              # Strike da opção vendida
K2 = 60              # Strike da opção usada para hedge
T = 1                # Tempo até o vencimento (anos)
r = 0.05             # Taxa livre de risco
sigma = 0.25         # Volatilidade
N_options = 1000     # Número de opções vendidas
dt = 1/12            # Um mês

# Preços de mercado
C1_0 = 6.1680        # Preço inicial da call ATM
C2_0 = 2.5127        # Preço inicial da call 60
C1_1 = 5.8611        # Preço da call ATM após 1 mês
C2_1 = 2.2591        # Preço da call 60 após 1 mês

# Funções para Black-Scholes
def bs_d1(S, K, T, r, sigma):
    return (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def bs_delta(S, K, T, r, sigma):
    d1 = bs_d1(S, K, T, r, sigma)
    return norm.cdf(d1)

def bs_gamma(S, K, T, r, sigma):
    d1 = bs_d1(S, K, T, r, sigma)
    return norm.pdf(d1) / (S * sigma * np.sqrt(T))

# Calculando gregas no tempo 0
delta_50 = bs_delta(S, K1, T, r, sigma)
gamma_50 = bs_gamma(S, K1, T, r, sigma)
delta_60 = bs_delta(S, K2, T, r, sigma)
gamma_60 = bs_gamma(S, K2, T, r, sigma)

# Sistema de equações:
# x + y * delta_60 = -N_options * delta_50   (delta hedge)
# y * gamma_60 = -N_options * gamma_50       (gamma hedge)

gamma_total = -N_options * gamma_50
y = gamma_total / gamma_60
x = -N_options * delta_50 - y * delta_60

# Avaliação do portfólio após 1 mês (preço do ativo permanece o mesmo)
# Preços de mercado após 1 mês
value_short_calls = -N_options * C1_1
value_long_calls = y * C2_1
value_stock = x * S
portfolio_1m = value_short_calls + value_long_calls + value_stock

# Valor do portfólio no tempo 0
initial_short_calls = N_options * C1_0
initial_long_calls = -y * C2_0
initial_stock = -x * S
portfolio_0 = initial_short_calls + initial_long_calls + initial_stock

# Lucro
profit = portfolio_1m - portfolio_0

# Resultados
{
    "delta_50": delta_50,
    "gamma_50": gamma_50,
    "delta_60": delta_60,
    "gamma_60": gamma_60,
    "shares_held": x,
    "calls_60_bought": y,
    "initial_portfolio_value": portfolio_0,
    "portfolio_value_after_1m": portfolio_1m,
    "profit": profit
}


{'delta_50': 0.627409464153284,
 'gamma_50': 0.030273586554705557,
 'delta_60': 0.34300112611322126,
 'gamma_60': 0.0294108726712101,
 'shares_held': -274.347030548085,
 'calls_60_bought': -1029.3331616895528,
 'initial_portfolio_value': 22471.75696278159,
 'portfolio_value_after_1m': -21903.81807297712,
 'profit': -44375.57503575871}